# Detection


In [1]:
import re
import os
import sys
import cv2

os.environ['PYTHONPATH'] += "/Users/cosmos/Desktop/parleg-alpha/RealTimeObjectDetection/Tensorflow/models"
sys.path.append( "/Users/cosmos/Desktop/parleg-alpha/RealTimeObjectDetection/Tensorflow/models")

import numpy as np
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from PIL import Image
from matplotlib.image import imread

#functions
def clean_ocr(data):
    for i in data:
        m = re.search("^[a-zA-Z0-9_]*$", i)
    if m==None:
        data.remove(i)
        
    data2 = [i for i in data if not i.isalpha()]

    data3 = []

    for k in data2:
        if len(k)==5:
            data3 = data3 + [k]
            
    return data3



/opt/homebrew/Caskroom/miniforge/base/envs/TOD/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/homebrew/Caskroom/miniforge/base/envs/TOD/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/opt/homebrew/Caskroom/miniforge/base/envs/TOD/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/homebrew/Caskroom/miniforge/base/envs/TOD/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/homebrew/Caskroom/miniforge/base/envs/TOD/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused b

In [2]:
CURRENT_WORKSPACE_NAME = '/workspace'
ROOT_PATH = '/Users/cosmos/Desktop/parleg-alpha/RealTimeObjectDetection'
WORKSPACE_PATH = ROOT_PATH+'/Tensorflow/'+CURRENT_WORKSPACE_NAME
SCRIPTS_PATH = ROOT_PATH+'/Tensorflow/scripts'
APIMODEL_PATH = ROOT_PATH+'/Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'
dataset = 'Valour3'
frameNo = '62'


category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-50')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [3]:
category_index

{1: {'id': 1, 'name': 'arrow'}, 2: {'id': 2, 'name': 'scheme'}}

In [4]:
# def detect_part(image_path):
    
#crop

coordinates = []
max_boxes_to_draw = 5
min_score_thresh=.3


img = imread("/Users/cosmos/Desktop/parleg-alpha/data/img_006.jpg")
# print(type(img))


image_np = np.array(img)
# print(type(image_np))
print(image_np.shape)

    # return imgout,coordinates

(1280, 960, 3)


In [5]:
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
print(input_tensor.shape)

(1, 1280, 960, 3)


In [6]:
detections = detect_fn(input_tensor)

2022-07-11 20:09:52.659861: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-11 20:09:52.671869: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [ ]:

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
            for key, value in detections.items()}
detections['num_detections'] = num_detections

# print(detections['detection_classes'])
# print(type(detections['detection_classes']))

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

imgout = Image.fromarray(image_np, 'RGB')

#cropping

boxes = detections['detection_boxes']
# max_boxes_to_draw = boxes.shape[0]
scores = detections['detection_scores']


for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    if scores[i] > min_score_thresh:
        class_id = int(detections['detection_classes'][i] + 1)
        box = boxes[i]
        im_height = imgout.height
        im_width = imgout.width
        (ymin, xmin, ymax, xmax) = (box[0] * im_height, box[1] * im_width,
                        box[2] * im_height, box[3] * im_width)
        coordinates.append({
            "box": [int(xmin) , int(ymin) , int(xmax) ,int(ymax)],
            "class_name": category_index[class_id]["name"],
            "score": scores[i]
        })

In [9]:
img,coordinates = detect_part("/Users/cosmos/Desktop/parleg-alpha/data/img_0010.jpg")

(1280, 960, 3)
(1, 1280, 960, 3)


InvalidArgumentError:  Input to reshape is a tensor with 162 values, but the requested shape requires a multiple of 91
	 [[node WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/Reshape_9 (defined at opt/homebrew/Caskroom/miniforge/base/envs/TOD/lib/python3.9/site-packages/object_detection/predictors/heads/keras_class_head.py:366) ]] [Op:__inference_detect_fn_45590]

Errors may have originated from an input operation.
Input Source operations connected to node WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/Reshape_9:
 WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/Identity_4 (defined at opt/homebrew/Caskroom/miniforge/base/envs/TOD/lib/python3.9/site-packages/object_detection/predictors/heads/keras_class_head.py:363)

Function call stack:
detect_fn


In [71]:
coordinates

[{'box': [762, 268, 861, 368], 'class_name': 'arrow', 'score': 0.99898094},
 {'box': [660, 157, 751, 248], 'class_name': 'scheme', 'score': 0.99289346}]

# Cropping

In [72]:
def translate_box(coordinates):
    left,upper,right,lower = coordinates[0]['box']
    for coordinate in coordinates:
        if coordinate['class_name']=="arrow":
            left,upper,right,lower = coordinate['box']
            new_upper = upper - (lower-upper)
            right = right + (right-left)*0.2
            lower = lower - (lower-upper)
            return (left,new_upper,right,lower)   

In [80]:
new_img = img.crop(translate_box(coordinates=coordinates))
new_img.save("/Users/cosmos/Desktop/parleg-alpha/production/final.png")

In [82]:
import easyocr
import re
import numpy

text_reader = easyocr.Reader(['en']) #Initialzing the ocr

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [91]:
def ocr(img_array):
    
    results = text_reader.readtext(img_array)
    data = []
    finall = []
    for (bbox, text, prob) in results:
        data = data + [text]

    for i in data:
        m = re.search("^[a-zA-Z0-9_]*$", i)
        if m==None:
            data.remove(i)
            
    final = [i for i in data if not i.isalpha()]
    return final,results

In [92]:
ocr(numpy.array(new_img))

(['Veu19'],
 [([[22, 12], [115, 12], [115, 45], [22, 45]], 'Veu19', 0.02697157250068769),
  ([[20, 44], [94, 44], [94, 72], [20, 72]], '144 , G0', 0.07553660693692277)])